<font size=3 color='blue'>
    
  # <center>Clase 2, marzo 10 del 2021</center> 

<font size=3 color='blue'>
    
  # <center>Ajuste de un conjunto de puntos con una función</center> 

<font size=2 color='blue'>
    
# Generación de las muestras

<font size=5 >

1.  Se genera un conjunto de numeros aleatorios ($(x_1,y_1),(x_2,y_2),…,(x_n,y_n)$)

In [ ]:
# Importing the python libraries

import numpy as np
import random
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

# To have a reproducible code we use a random seed 
np.random.seed(3)

In [ ]:
# Esta función genera un conjunto de datos que simulan 
# la medición de la distancia de un carrito en un riel de aire
# en la ausencia de una fuerza sobre el carrito.
# Se propone un error en la medición de la distancia

def generador_datos_simple(n_points, distance_0, measure_time, speed, max_distance_error):
    
    # n_points es el número de puntos que serán generados
    
    x = np.random.random(n_points) * measure_time
     
    # x es arreglo con m numeros aleatorios entre 0.0 y measure_time
    
    error = np.random.randn(n_points) * max_distance_error 
    
    # e es un error generado aleatoriamente con un valor maximo max_distance_error

    y = distance_0 + speed*x + error 
        
    return x.reshape((n_points,1)), y.reshape((n_points,1))


In [ ]:
# Generacción de las muestras (x,yi)
n_points = 500
distance_0 = 100.0
measure_time = 100.0
speed = 20.0
max_distance_error = 100

x, y = generador_datos_simple(n_points, distance_0, measure_time, speed, max_distance_error)

print("x type", type(x), "x shape", x.shape)
print("y type", type(y), "y shape", y.shape)

In [ ]:
x = np.squeeze(x)
y = np.squeeze(y)
print("x type", type(x), "x shape", x.shape)
print("y type", type(y), "y shape", y.shape)

In [ ]:
# Plotting y versus x

plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)

plt.scatter(x, y)
plt.show()

<font size=2 color='blue'>

# Buscando la correlación entre las muestras:



<font size=4>

Se tiene un conjunto de m muestras (puntos medidos) $(x_i, y_i)$, y se busca encontrar una función F que describa una posible correlación entre ellas.

Planteamos que los valores de $y_i$ dependen de los valores de $x_i$. 

Para encontrar esta función, inicialmente se propone un conjunto de funciones definidas todas ellas por misma relación funcional de un conjunto de parámetros (variables). Cada función es definida por valores específicos de estos parámetros.

Por ejemplo, la relación funcional $$f(x) = b + w x$$ $$ $$define de manera general a todas las funciones que describen una recta en el plano $x-y$. $$ $$En especial la función 

$$f = 0.0 + 2.0 x$$


define a una recta que corta al eje $y$ en 0.0 y tiene una pendiente de 2.0. 

Mientras que la función $$f = 10.0 + 8.0x,$$ 
$$ $$ define una recta que corta al eje $y$ en 10.0 y tiene una pendiente de 8.0.

En el presente problema, para encontrar la correlación entre las muestras, proponemos un conjunto de funciones definidas mediante la siguiente relación lineal:

$$ f(x, w, b) = b + w x $$
  
Vemos que esta relación funcional es derivable respecto a todas sus variables, x, w, b.
La letra w se emplea como abreviación de la palabra en ingles "weight". El valor de w nos dice el valor que la variable x tiene en el valor de la función f. La letra b es la abreviación de la palabra "bias" en ingles, y se refiere al valor que toma la función f para el valor de x = 0.


<font size = 4>

Para encontrar la función f que mejor describe la correlación entre los puntos, es necesario generar una métrica para describir que tanto se acerca cada una de las funciones específicas (con valores definidos de w y de b) a esta descripción.

La métrica que se propone es la siguiente: $$ $$

Dados los valores de b y w, para cada muestra $(x_i, y_i)$ se evalua $f(w, b, x_i)$ y se compara con el correspondiente valor medido $y_i$. La diferencia entre estos valores se eleva al cuadrado. 
    
$$ (f(w,b,x_i)-y_i)^{2}$$


Finalmente se calcula el promedio de este valor sobre todas las muestras, el cual definimos como residuo.

Si m es el número de muestras, el residuo queda como:

$$ Residuo = \dfrac {1}{m}∑_{i=1}^{m}(f(w,b,x_i)-y_i)^{2} $$

<font size=5, color='blue'>
    
En el siguiente código se implementa la generación del residuo dada una función
    
específica definido por los pesos iniciales w = weight_0, y b = bias_0.

In [ ]:
#Initializing the variables of the function f

weight_0 = 10.0
bias_0 = 200.0

print(weight_0, bias_0)

In [ ]:
# Function for plotting (x,y) and (x,y_)

def plot_x_y_y__(x, y, y_, points=True):
    
    plt.figure(figsize=(13,8))
    plt.rc('xtick', labelsize=16)
    plt.rc('ytick', labelsize=16)
    plt.rc('legend', fontsize=16)
    plt.ylabel('Y', fontsize=16)
    plt.xlabel('X', fontsize=16)

    #Plotting function
    plt.plot(x, y_, color='green', lw=4, label='F(x, w, b)')
    plt.legend()

    #Plotting samples
    plt.scatter(x, y)

    plt.show()
    
    

<font size=4>

Se grafica la correspondiente función f para los valores $x_i$, así como las muestras $(x_i, y_i)$

In [ ]:
#The following arrays are generated for plotting the Function F(x, weight_0, bias_0)

y_ = weight_0*x + bias_0

# Using the function f, the residuos is calculated by comparing the calculated and measured values

residuo = np.mean((y-y_)**2)

print('residuo: {0:10.2f}'.format( residuo))

plot_x_y_y__(x, y, y_, points=True)

<font size=4>

Como se observa en la grafica, esta función está lejos de describir la correlación entre los puntos. 


<font size=5 color="blue">

Los valores del peso w y el bias b se actualizan iterativamente a prueba y error. Para ello

se deben hacer los cambios de acuerdo a la gráfica que se obtenga. 

In [ ]:
def update_weights_biases(x, y, weight, bias, delta_weight, delta_bias):
    
    weight = weight + delta_weight
    
    bias = bias + delta_bias

    #The following date are for constructing the F(x,weight, bias)
    
    y_ = weight*x + bias
       
    residuo = np.mean((y - y_)**2)
    
    #print('residuo: {:10.2f}'.format(residuo))   
        
    return weight, bias, y_, residuo

In [ ]:
weight = weight_0
bias = bias_0
delta_w = 1.0
delta_b = -10.0

weight, bias, y_, residuo = update_weights_biases(x, y , weight, bias, delta_w, delta_b)
plot_x_y_y__(x, y, y_, points=True) 

<font size=4>

El cambio es muy pequeño, por ello el peso se actualiza con delta_weight = 2.0 y el bias con delta_bias= -100.0. 

In [ ]:
weight = weight
bias = bias
delta_w = 6.0
delta_b = -100.0

weight, bias, y_, residuo = update_weights_biases(x, y, weight, bias, delta_w, delta_b)
plot_x_y_y__(x, y, y_, points=True) 

<font size=4>

Aun se puede mejorar disminuyendo el peso (pendiente) y aumentando el bias (ordenada en el origen):

In [ ]:
weight = weight
bias = bias
delta_w = 1.5
delta_b = 50.0

weight, bias, y_, residuo = update_weights_biases(x,y, weight, bias, delta_w, delta_b)
plot_x_y_y__(x, y, y_, points=True) 

<font size=4>

Se puede mejorar el resultado. Para ello se observa el cambio en el residuo con los cambios en w y en b.

<font size=5 color='blue'>
    
Busquemos encontrar la función que defina la correlación generando un conjunto de
    
funciones y calcular los correspondientes residuos.

<font size=4>

Para ello emplearemos la función **update_weight_bias()**, la cual actualiza los parametros weight y bias.

In [ ]:
parameters = []

weight = weight_0
bias = -3.0
delta_w = 0.2 
delta_b = 0.0

for i in range(100):
    
    weight, bias, y_, residuo = update_weights_biases(x, y, weight, bias, delta_w, delta_b)
    
    parameters.append([weight, bias, y_, residuo])
    
    if i % 10 == 0 :
            print('weight: {0:10.2f} bias: {1:10.2f} residuo: {2:10.2f}'.format(weight,bias, residuo))

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=12)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)

for i in range(0,50,3):

    plt.plot(x, parameters[i][0]*x + parameters[i][1], label='v' + str(i), lw=3)
    plt.legend()

plt.scatter(x, y)

plt.show()

<font size=4>

A continuación se grafica el residuo en función de la pendiente, w.
 
 El valor óptimo de la pendiente es aquel que minimiza al residuo. 

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Residuo', fontsize=16)
plt.xlabel('W', fontsize=16)

weights = []

residuos = []

for i in parameters:
    
    weights.append(i[0])
    
    residuos.append(i[3])
    
plt.scatter(weights, residuos)


plt.show()

<font size = 4>

Como se observa en la grafica anterior, el mínimo de la pendiente esta cerca de w = 22.

In [ ]:
index_min = np.argmin(residuos)   # returns the index for the minimum value in the list

print('El residuo mas pequeño tiene el valor: {0:10.2f}'.format(parameters[index_min][3]))

w_opt = parameters[index_min][0]
b_opt = parameters[index_min][1]

print('Los valores optimos de los parámetros son w = {0:5.2f} y b = {1:5.2f} ' \
      .format(w_opt, b_opt))

<font size = 4>

Se grafica la recta con los valores óptimos encontrados w_opt and b_opt :

In [ ]:
y_ = w_opt*x + b_opt

plot_x_y_y__(x, y, y_, points=True)

<font size=5 color='blue'>

Se generan un conjunto de funciones dejando w fija y variando b.

In [ ]:
parameters = []

weight =  w_opt
bias = b_opt
delta_w = 0.0
delta_b = 1.0

for i in range(100):
    
    weight, bias, y_, residuo = update_weights_biases(x, y, weight, bias, delta_w, delta_b)
    
    parameters.append([weight, bias, y_, residuo])
    
    if i % 10 == 0 :
        print('weight: {0:10.2f} bias: {1:10.2f} residuo: {2:10.2f}'.format(weight,bias,residuo))

In [ ]:
plt.figure(figsize=(13,12))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=12)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)

for i in range(0,50,3):

    plt.plot(x, w_opt*x + parameters[i][1], label='v' + str(i), lw=3)
    plt.legend()

plt.scatter(x, y)

plt.show()

<font size=4>

A continuación se grafica el residuo en función de b.
 
 El valor óptimo de b es aquel que minimiza al residuo. 

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Residuo', fontsize=16)
plt.xlabel('b', fontsize=16)

bias = []

residuos = []

for i in parameters:
    
    bias.append(i[1])
    
    residuos.append(i[3])
    
plt.scatter(bias, residuos)

plt.show()

<font size = 4>

Como se observa en la grafica el mínimo del residuo corresponde a un valor de b cercano a 20.

In [ ]:
index_min = np.argmin(residuos)   # returns the index for the minimum value in the list
residuo_min = parameters[index_min][3]

print('El residuo mas pequeño tiene el valor: {0:10.2f}'.format(residuo_min))

w_opt = parameters[index_min][0]
b_opt = parameters[index_min][1]

print('Los valores optimos de los parámetros son w = {0:5.2f} y b = {1:5.2f} ' \
      .format(w_opt, b_opt))

<font size = 4>

Se grafica la recta empleando los valores óptimos w, w_opt, y b, b_opt.

In [ ]:
y_ = w_opt*x + b_opt
y_hand = y_

plot_x_y_y__(x, y, y_, points=True)

In [ ]:
type(residuos)

In [ ]:
residuos_array = np.asarray(residuos)
type(residuos_array)

In [ ]:
residuos_grad = np.gradient(residuos_array)
residuos_grad

In [ ]:
plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Derivada del Residuo', fontsize=16)
plt.xlabel('b', fontsize=16)

bias = []

residuos = []

for i in parameters:
    
    bias.append(i[1])
    
    residuos.append(i[2])

plt.grid(True)
plt.scatter(bias, residuos_grad)

plt.show()

<font size=4>

En este caso, el residuo es una parábola, por tanto su derivada es una recta. En general el residuo tendrá una forma muy diferente al de una parábola

<font color = 'blue' size = 4>

### Método simple de mínimos cuadrados, de acuerdo a Gauss

<font size=4>

El método de mínimos cuadrados nos permite encontrar una función que describe la correlación que tienen un conjunto m de puntos $(x_i, y_i)$.


El objetivo del algoritmo es obtener la relación entre la variables independiente x, y la variable dependiente y.

En el presente caso, se propone una función linear para describir la correlación:

$$ F(x_i) = b + w x_i $$
 

Los parámetros variables b  y w definen a la función F. 

Con el método simple de mínimos cuadrados se hallan los valores óptimos w_opt y b_opt de estas variables.

<font size = 4>

La diferencia entre el valor real $y_i$ y el valor estimado $F(x_i)$ se denomina ($r_i$):

$$ r_i = y_i - F(x_i) $$

El objetivo del método de mínimos cuadrados es minimizar la suma del cuadrado de los residuos, es decir, minimizar la función Residuo:

$$ Residuo = \dfrac {1}{m} ∑_{i=1}^{m}(y_i - b -wx_i)^{2} $$

Derivando parcialmente con respecto a los parámetros w y b, e igualando la función a cero, se obtiene:

$$ \dfrac{∂Residuo}{∂\hat{w}} =0⟶ w_{opt}=\dfrac{∑_{i=1}^{n}(x_i–\bar x)(y_i–\bar y)}{∑_{i=1}^{m}(x_i–\bar x)^2}$$


$$ \dfrac{∂Residuo}{∂\hat{b}}=0⟶ b_{opt}=y– w_{opt} x $$

en donde $\bar x$ y $\bar y$ son los promedios de las muestras $(x_i, y_i)$.



In [ ]:
def mean_square_error(x, y):
    
    # 1) Se obtiene el promedio de los valores x_i y y_i

    mean_x = np.mean(x)

    mean_y = np.mean(y)

    # 2) se calcula (x_i-x) y (y_i-y), con x e y los promedios de x_i e y_i, respectivamente.

    x_i = []

    for i in x:

        x_i.append(np.squeeze(i) - mean_x)

    y_i = []

    for i in y:

        y_i.append(np.squeeze(i) - mean_y)

    # 3) se calcula (x_i-x)*(x_i-x)

    x_i2 = np.power(x_i, 2)

    # 4) se hacen las sumas correspondientes

    xy_sum = 0

    for i in range(len(x_i)):

        xy_sum += x_i[i]*y_i[i]

    x2_sum = 0

    for i in range(len(x_i)):

        x2_sum += x_i2[i]

    # Se definen a w y b 

    w = xy_sum/x2_sum

    b = mean_y - w*mean_x
       
    return w, b



In [ ]:

w_opt_mse, b_opt_mse = mean_square_error(x, y)

print(" w_opt = {0:5.2f}, b_opt = {1:5.2f}".format(w_opt_mse,b_opt_mse))

In [ ]:
y_ = w_opt_mse*x + b_opt_mse

plot_x_y_y__(x, y, y_, points=True)

In [ ]:

y_MSE = x*w_opt_mse + b_opt_mse

plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)

plt.plot(x, y_MSE, color='green', lw=4)

plt.scatter(x, y)

plt.show()

In [ ]:
MSE = np.mean((y-w_opt*x-b_opt)**2)
print('MSE: {0:10.2f}'.format(MSE))

In [ ]:
y_MSE = x*w_opt + b_opt

plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)


plt.plot(x, y_hand, color='magenta', label='hand_MSE-residuo', lw=6)

plt.plot(x, y_MSE, color='blue', label = 'Gauss_MSE-residuo', lw=3)

plt.legend()

plt.scatter(x, y)

plt.show()


<font size=5 color='blue'>
    
Método de Newton-Raphson, para encontrar el cero de una función. 
    
Este método emplea la función y su derivada.
    


<font size=4>

El código que generaremos para emplear este método requiere un valor inicial de la variable que define la función, así como una epsilon para verificar si ya se considera que encontró el cero. El método tambien considera el número máximo de iteraciones que puede efectuar.

<img src = "NewtonIteration_Ani.gif">

(By Ralf Pfeifer - de:Image:NewtonIteration Ani.gif, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=2268473)

In [ ]:
def newton_raphson(f,Df,x0,epsilon,max_iter):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        
    Df : Derivative of f(x).
    
    x0 : Initial guess for finding the root of f(x).
    
    epsilon :Stopping criteria: the iteration ends when abs(f(x)) < epsilon.
    
    max_iter : Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.
    '''
    
    aprox_root = [x0]
    
    xn = x0
      
    # xn es la aproximación de la raíz de f. Inicialmente xn =x0 con x0 la primera aproximación
    
    for n in range(0,max_iter):
        
        fxn = f(xn)
        print("xn = ", xn, "aprox_root = ", aprox_root)
        
        if abs(fxn) < epsilon:
            
            print('Found solution after',n,'iterations.')
            
            return xn, aprox_root
        
        Dfxn = Df(xn)
        
        if Dfxn == 0:
            
            print('Zero derivative. No solution found.')
            
            return None
        
        print("fxn = ", fxn, "Dfxn = ", Dfxn)
        
        xn = xn - fxn/Dfxn
        
        aprox_root.append(xn)
        
    print('Exceeded maximum iterations. No solution found.')
    
    return None

<font size=4>

Se prueba la método con una función simple

In [ ]:
#p = lambda x: x**3-27

def p(x):
    
    return x**3-27

In [ ]:
for i in range(5):
    
    print(p(i))

In [ ]:
x_ = np.arange(-10, 10)

y_ = p(x_)

plt.figure(figsize=(13,8))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)
plt.ylabel('Y', fontsize=16)
plt.xlabel('X', fontsize=16)


plt.grid(True)

plt.plot(x_, y_, color='r')

plt.show()

print(y_)

In [ ]:
#Dp = lambda x: 3*x**2 

def Dp(x):
    
    return 3*x**2

newton_raphson(p,Dp,7.5,0.001,50)

In [ ]:
def tangent_line(f, Df, x_0, a, b):
        
    x = np.linspace(a,b)
    
    y = f(x) 
    
    y_0 = f(x_0)
    
    y_tan = Df(x_0) * (x - x_0) + y_0 
    
    plt.figure(figsize=(13,8))
    plt.rc('xtick', labelsize=16)
    plt.rc('ytick', labelsize=16)
    plt.rc('legend', fontsize=16)
    plt.ylabel('Y', fontsize=16)
    plt.xlabel('X', fontsize=16)



    plt.plot(x,y,'r-')
    
    plt.plot(x,y_tan,'b-')
    
    plt.xlabel('x') 
    
    plt.ylabel('y') 
    
    plt.grid(True)
    
    plt.title('Plot of a function with tangent line') 
    
    plt.show()  

In [ ]:

for i in newton_raphson(p,Dp,7.5,0.001,50)[1]:

    tangent_line(p, Dp, i, -10, 10)

<font size=4>

Para aplicar este método en nuestro caso, es necesario implementarlo para 2 variables (que en nuestro caso son w y b).

El método en general ha sido implementado para problemas con multivariables (número de variables mayor a 2)

[Newton-Raphson Multivariate](./Literatura/Newton-Raphson-multivariate.pdf)

<font size=4>

Otra alternativa para encontrar los valores de w y b para los cuales el residuo es mínimo es emplear un método iterativo desarrollado por Cauchy en el siglo XIX. Hoy en día este método de optimización es conocido como el método de gradiente descendente.
    
[Cauchy, Gradiente descendente](./Literatura/Cauchy_gradient-descent.pdf)
    
En este caso durante la optimización, los valores de w y b cambian su valor disminuyéndolo con el gradiente del residuo multiplicado par el factor alfa (> 0).
    
 w_final = w_inicial - alfa * Gradiente(Residuo)_w
    
 b_final = b_inicial - alfa * Gradiente(Residuo)_b